In [10]:
# # CSV to VTK

# import pandas as pd
# import vtk
# import os

# # 设置CSV文件所在的文件夹路径
# folder_path = r"C:\Users\14014\Downloads\000"

# # 遍历文件夹中的所有CSV文件
# for filename in os.listdir(folder_path):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(folder_path, filename)

#         # 读取CSV文件
#         df = pd.read_csv(file_path)

#         # 创建一个vtkPoints对象存储点数据
#         points = vtk.vtkPoints()
#         for index, row in df.iterrows():
#             points.InsertNextPoint(row['x'], row['y'], row['z'])

#         # 创建一个vtkPolyLine对象，它表示一系列线段，其中所有点都按顺序连接
#         polyLine = vtk.vtkPolyLine()
#         polyLine.GetPointIds().SetNumberOfIds(len(df))
#         for i in range(len(df)):
#             polyLine.GetPointIds().SetId(i, i)

#         # 创建一个vtkCellArray来存储这些线段
#         cells = vtk.vtkCellArray()
#         cells.InsertNextCell(polyLine)

#         # 创建一个vtkPolyData对象
#         polyData = vtk.vtkPolyData()
#         polyData.SetPoints(points)
#         polyData.SetLines(cells)

#         # 为点添加额外的数据，如curvature和torsion
#         curvature = vtk.vtkDoubleArray()
#         curvature.SetName("curvature")
#         torsion = vtk.vtkDoubleArray()
#         torsion.SetName("torsion")
#         for index, row in df.iterrows():
#             curvature.InsertNextValue(row['curvature'])
#             torsion.InsertNextValue(row['torsion'])
#         polyData.GetPointData().AddArray(curvature)
#         polyData.GetPointData().AddArray(torsion)

#         # 将vtkPolyData写入VTK文件
#         writer = vtk.vtkPolyDataWriter()
#         writer.SetFileName(file_path.replace('.csv', '.vtk'))
#         writer.SetInputData(polyData)
#         writer.Write()


INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO: NumExpr defaulting to 8 threads.


In [23]:
# 【不动】头文件和函数定义
import sys
sys.path.append('D:\\bai_work_win\\AngioMorph\\src')

import numpy as np 
import glob 
import vtk
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from AngioMorphPCA.preprocessing import parameterize_curve, calculate_3d_curve_abscissas,remove_high_freq_components
from AngioMorphPCA.io import Get_simple_vtk, makeVtkFile, mkdir
from scipy.integrate import simps
from AngioMorphPCA.GenerateDiffHemeo import generate_diff_homeomorphism
from AngioMorphPCA.srvf_rep import compute_srvf_func, reconstruct_curve_from_srvf
from AngioMorphPCA.L2distance import calculate_l2_distance
from AngioMorphPCA.compute_geometry import compute_curvature_and_torsion,build_curve_from_curvatures
from tqdm import tqdm
import matplotlib.pyplot as plt
from geomstats.learning.frechet_mean import FrechetMean
import geomstats.backend as gs
from geomstats.geometry.discrete_curves import (
    DiscreteCurvesStartingAtOrigin,
    SRVMetric,
    insert_zeros,
)

def output_vtk_file(shape, output_file):
    # 创建VTK点数据和折线连接信息
    points = vtk.vtkPoints()
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(shape.shape[0])  # 设置点的数量

    # 插入平均形状的点，并添加点到折线
    for i, point in enumerate(shape):
        points.InsertNextPoint(point.tolist())
        polyline.GetPointIds().SetId(i, i)

    # 创建包含折线的vtkCellArray
    lines = vtk.vtkCellArray()
    lines.InsertNextCell(polyline)

    # 创建vtkPolyData对象来保存点和连接信息
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(points)
    poly_data.SetLines(lines)

    # 写入VTK文件（使用旧式的VTK文件格式）
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(output_file)
    writer.SetInputData(poly_data)
    writer.Write()

    print(f"形状的VTK文件已保存至: {output_file}")



def compute_centroid(curves):
    centroid = np.mean(curves, axis=0)
    return np.array(centroid)
def translate_to_centroid(curves):
    centroid = compute_centroid(curves)
    new_curves = []
    for i in range(len(curves)):
        new_curves.append(curves[i] - centroid)
    return np.array(new_curves)

def plot_curve(curve, ax=None, add_origin=True):
    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection="3d")

    if add_origin:
        curve = insert_zeros(curve, axis=-2)

    ax.plot(*[curve[:, k] for k in range(3)])
    ax.scatter(*[curve[0, k] for k in range(3)])
    return ax

def plot_geodesic(geod_points, ax=None, add_origin=True):
    n_times = geod_points.shape[0]
    k_sampling_points = geod_points.shape[-2] + 1

    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection="3d")

    if add_origin:
        geod_points = insert_zeros(geod_points, axis=-2)

    ax.plot(*[geod_points[0, :, k] for k in range(3)],  c="b", linewidth=2)
    ax.plot(*[geod_points[-1, :, k] for k in range(3)], c="r", linewidth=2)

    for i in range(1, n_times - 1):
        ax.plot(*[geod_points[i, :, k] for k in range(3)], c="k", linewidth=1)

    for j in range(k_sampling_points - 1):
        ax.plot(*[geod_points[:, j, k] for k in range(3)], c="k", linewidth=1)

    return ax


In [32]:
files = glob.glob('uezono_data/*.vtk')

Len = len(files)

print(files)

resample_num=120

curves = []
for i in range(Len):
    casename = files[i].split('\\')[-1].split('.')[0]
    temp = Get_simple_vtk(files[i])

    # 调换index
    temp = temp[::-1]

    temp = translate_to_centroid(temp)
    temp_func = parameterize_curve(temp) # 重参数化，归一化，成了单位长度?(看输出不是)
    t_resampled = np.linspace(0, 1, resample_num) # 重采样，生成一个从 0 到 1 的均匀分布
    resampled_curve = temp_func(t_resampled) # 用重采样的 t 值生成新的曲线
    curves.append(resampled_curve)

# np.save("unaligned_curvatures.npy", curvatures)
mean_curve = np.mean(curves, axis=0)


['uezono_data\\sub01-left.vtk', 'uezono_data\\sub01-right.vtk', 'uezono_data\\sub02-left.vtk', 'uezono_data\\sub02-right.vtk', 'uezono_data\\sub03-left.vtk', 'uezono_data\\sub03-right.vtk']


In [29]:

output_file = "D:\\bai_work_win\\AngioMorph\\bai_output\\uezono_data1\\"
output_vtk_file(mean_curve, output_file + 'mean_curve_resampled.vtk')
for i in range(Len):
    output_vtk_file(curves[i], output_file + files[i].split('\\')[-1].split('.')[0] + '_resampled.vtk')

形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\mean_curve_resampled.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub01-left_resampled.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub01-right_resampled.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub02-left_resampled.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub02-right_resampled.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub03-left_resampled.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub03-right_resampled.vtk


In [33]:
 # geomstats归一化，对齐

k_sampling_points = resample_num

curves_r3 = DiscreteCurvesStartingAtOrigin(
    ambient_dim=3, k_sampling_points=k_sampling_points, equip=False
) #初始化曲线空间
curves_r3.equip_with_metric(SRVMetric) # 配置SRV度量


curve_mean = curves_r3.projection(mean_curve)
curve_mean = curves_r3.normalize(curve_mean) # 缩放为单位长度



curve_bs = []
# curve_bs.append(curve_mean)

for i in tqdm(range(1,len(curves))):
# for i in range(4):
    curve_b = curves_r3.projection(curves[i])
    curve_b = curves_r3.normalize(curve_b)

    curves_r3.equip_with_group_action("rotations and reparametrizations")
    curves_r3.equip_with_quotient_structure()

    # 先和简单平均对齐了一下
    curve_b_aligned = curves_r3.fiber_bundle.align(curve_b, curve_mean)
    curve_bs.append(np.array(curve_b_aligned))

    # hgeod_fun = curves_r3.quotient.metric.geodesic(curve_a, curve_b)

    # n_times = 10
    # times = gs.linspace(0.0, 1.0, n_times)
    # hgeod = hgeod_fun(times)

curve_bs = np.array(curve_bs)



100%|██████████| 5/5 [00:08<00:00,  1.77s/it]


In [34]:


output_file = "D:\\bai_work_win\\AngioMorph\\bai_output\\uezono_data1\\"

output_vtk_file(curve_mean, output_file + 'mean_curve_normed.vtk')

for i in range(len(curve_bs)):
    output_vtk_file(curve_bs[i], output_file + files[i].split('\\')[-1].split('.')[0] + '_aligned.vtk')

形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\mean_curve_normed.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub01-left_aligned.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub01-right_aligned.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub02-left_aligned.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub02-right_aligned.vtk
形状的VTK文件已保存至: D:\bai_work_win\AngioMorph\bai_output\uezono_data1\sub03-left_aligned.vtk
